AllLife  bank interesteed to grow their business into loan sectors as well as keeping their deposit buisness. The main objective of this project is to identify the variables which has more influences on Personal_Loan variable. There are given different features, such as Age, Income, Experience, Education, ID, ZIPCode, CreditCard, Online, Securities_Account, CD_Account etc. Based on the analysis, which segmets are more lucrative to target or carry more probability to convert into asset customer. 
A short description of different features relevant to the project are given below:
ID: Customer ID
Age: Customer's age
Experience: Years of professional experience
Income: Annual income of the customer (in thousand dollars)
ZIP Code: Home Address ZIP Code
Family: Family size of the customer
CCAvg: Average spending on credit cards per month (in thousand dollars)
Education: Education Level. 1: Undergrad; 2: Graduate;3: Advanced/Professional
Mortgage: Value of house mortgage if any. (in thousand dollars)
Personal_Loan: Did this customer accept the personal loan offered in the last campaign?
Securities_Account: Does the customer have securities account with the bank?
CD_Account: Does the customer have a certificate of deposit (CD) account with the bank?
Online: Do customers use internet banking facilities?
CreditCard: Does the customer use a credit card issued by any other Bank (excluding All life Bank)?

# Importing necessary libraries

In [1]:
%load_ext nb_black

ModuleNotFoundError: No module named 'nb_black'

In [ ]:
#Library to supress warning or depracation notes
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Libraries to read, load and manipulate the data
import pandas as pd
import numpy as np

In [5]:
#Libraries to visualize the data
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [6]:
#Removing the limit for column and setting the limit for row to 200
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

In [22]:
#Library to build linear regression
from sklearn.linear_model import LogisticRegression
#Libraries to build the decision tree classifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#Library to tune different models
from sklearn.model_selection import GridSearchCV
#lIbrary to split the data into train and test data
from sklearn.model_selection import train_test_split

In [23]:
# to get different metrics score
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
    precision_recall_curve,
    make_scorer,
    roc_curve,
    confusion_matrix,
)

# Loading the dataset

In [8]:
#loading the dataset
loanmo=pd.read_csv("Loan_Modelling.csv")

In [9]:
#copying the dataset and renaming for future use
loanc=loanmo.copy()

In [ ]:
#checking the first 10 rows of the data
loanc.head(10)

In [ ]:
#Checking the last 10 rows of the data
loanc.tail(10)

In [ ]:
loanc.shape

The dataset has 5000 rows and 14 columns

In [ ]:
#The column names of the datasets
loanc.columns

In [ ]:
# Checking the datatypes
loanc.info()

Almost all columns are integer type only except ccavg, which is float data. ID and ZIPCode are also integer, which should be converted into suitable data types or to drop, which will be decided in later section. Also Education, Personal_Loan, Securities_Account, CD_Account, Online, Credit Card are binary data types, they should be converted into categorical data types. 

In [ ]:
cat_cols=["Personal_Loan", "Securities_Account", "CD_Account", "Online", "CreditCard", "Education"]
loanc[cat_cols] = loanc[cat_cols].astype("category")
loanc.info()


Almost all columns are integer type only except ccavg, which is float data. ID and ZIPCode are also integer, which should be converted into suitable data types or to drop, which will be decided in later section

In [ ]:
loanc.describe().T

-Only very percentage of customer has personal loan, securities account, and CD_Account with the bank
- Average income of the customer is approximately 73000 dollar
-On average customers have 20 years of working experience
- In experience column, there are negative value is also presence, which is unrealistic. However, we can assume, these are typo, Maybe mistakenly instead of postive values were negative values typed. 
- ID will have all unique values
- ZIPCode will also have many unique values
- Personal_Loan, Securities_Acccount, CD_Account will have probably right skewed data


In [ ]:
loanc=loanc.drop(["ID"], axis=1)

In [ ]:
loanc.head()

In [ ]:
#Checking for null data
loanc.isnull().any()

There are no missing values

In [ ]:
loanc.duplicated()

There are no duplicated values

In [ ]:
loanc.describe(include="category").T

In [ ]:
for i in cat_cols:
    print("Uniques Values in", i, "are :")
    print(loanc[i].value_counts()/sum(loanc[i].value_counts())*100)
    print("*"*50)

# Univariate Analysis

In [ ]:
# defining function for distribution plot and boxplot together
def distplot_boxplot(feature, figsize=(15, 10), kde=False, bins=None):

    f2, (ax_box2, ax_dist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        feature, ax=ax_box2, showmeans=True, color="yellow"
    )  # boxplot will be created and a star will indicate the mean value of the column
    sns.distplot(
        feature, kde=kde, ax=ax_dist2, bins=bins, color="green"
    ) if bins else sns.distplot(
        feature, kde=kde, ax=ax_dist2
    )
    ax_dist2.axvline(np.mean(feature), color="red", linestyle="--")
    ax_dist2.axvline(np.median(feature), color="black", linestyle="-")
    ax_box2.axvline(np.mean(feature), color="red", linestyle="--")
    ax_box2.axvline(np.median(feature), color="black", linestyle="-")

In [ ]:
distplot_boxplot(loanc.Age, bins=25);

-The age of the customers are almost normally distributed
- Average age is around 45 

In [ ]:
distplot_boxplot(loanc.Experience, bins=25);

There is no ouliers. Average experience is aorund 20. However, there are unrealistic negative values

In [ ]:
#checking the unique negative values
loanc[loanc["Experience"] < 0]["Experience"].unique()

In [ ]:
#replacing the negative values with positive values
loanc["Experience"].replace(-1,1, inplace=True)
loanc["Experience"].replace(-2,2, inplace=True)
loanc["Experience"].replace(-3,3, inplace=True)

In [ ]:
loanc["Experience"].describe().T

There is now 0 is minimum value

In [ ]:
distplot_boxplot(loanc.Income, bins=25);

- The distribution is rightly skewed and there are outliers in the data

In [ ]:
distplot_boxplot(loanc.Family, bins=25);

Average family size is 2.5

In [ ]:
distplot_boxplot(loanc.CCAvg, bins=25);

The distribution is right skewed and there are outliers in the data

Average Customer have accomplished their undergraduate level of education

In [ ]:
distplot_boxplot(loanc.Mortgage, bins=25);

Distribution is right skewed. 
there are many outliers in the data. 
Most of the customers have no mortgage.

Distribution is right skewed. That means most of the customer have not taken personal loan service.

The same kind of distribution also found in this case, so very few of cusomters have this account

The distribution is right skewed and most of the customer have no CD Account with the bank.

More customers have online presence.

Less customers use the credit card. 

In [ ]:
distplot_boxplot(loanc.CCAvg, bins=25);

Data is right skewed. Average customer spend below 2000 dollars per month on credit card. There are also many outliers in the data

In [ ]:
# A function is created for labeled barplots

def labeled_barplot(data, feature, perc=False, n=None):
    """
    Barplot with percentage at the top

    data: dataframe
    feature: dataframe column
    perc: whether to display percentages instead of count (default is False)
    n: displays the top n category levels (default is None, i.e., display all levels)
    """

    total = len(data[feature])  # length of the column
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count + 2, 6))
    else:
        plt.figure(figsize=(n + 2, 6))

    plt.xticks(rotation=90, fontsize=15)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Paired",
        order=data[feature].value_counts().index[:n].sort_values(),
    )

    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
                100 * p.get_height() / total
            )  # percentage of each class of the category
        else:
            label = p.get_height()  # count of each level of the category

        x = p.get_x() + p.get_width() / 2  # width of the plot
        y = p.get_height()  # height of the plot

        ax.annotate(
            label,
            (x, y),
            ha="center",
            va="center",
            size=12,
            xytext=(0, 5),
            textcoords="offset points",
        )  # annotate the percentage

    plt.show()  # show the plot

In [ ]:
labeled_barplot(loanc, "Personal_Loan", perc=True)

Only 9.6 percen of customer has taken personal loan

In [ ]:
labeled_barplot(loanc, "Securities_Account", perc=True)

Only 10.4% customers have used securities account

In [ ]:
labeled_barplot(loanc, "CD_Account", perc=True)

Only 6 percent customers have CD Account

In [ ]:
labeled_barplot(loanc, "Online", perc=True)

Around 60 percent of customer have online presence. 

In [ ]:
labeled_barplot(loanc, "CreditCard", perc=True)

Around 30 percent of customer have credit card other than AllLife Bank.

In [ ]:
labeled_barplot(loanc, "Family", perc=True)

Highest percent of the customer are single. 

In [ ]:
loanc["Education"].replace(1, "Undergraduate", inplace=True)
loanc["Education"].replace(2, "Graduate", inplace=True)
loanc["Education"].replace(3, "Postgraduate", inplace=True)
labeled_barplot(loanc, "Education", perc=True)

Maximum percentage of customers have undergrade level of education

In [ ]:
#Checking the unique value in zipcode
loanc["ZIPCode"].nunique()

-There are too many unique values
-These will not help to gain any insights for the bank
-I would suggest to drop the column

# Bivariate Analysis

In [ ]:
#Checking the correlations among different variables
locorr=loanc.corr()
print(locorr)

In [ ]:
#Plottig the heatmap based on the correlation
plt.figure(figsize=(15, 7))
sns.heatmap(locorr, annot=True, vmin=-1, vmax=1, cmap="Spectral")
plt.show()

In [ ]:
sns.pairplot(data=loanc, hue="Personal_Loan")
plt.show()

Observations from correlation, pairplot and heatmap:
- Age and experice are highly correlated with one another. In that case, we will keep only one variable between two variables. Considering the industry perspective, it is better to keep the experience variable, which might better can define the probability of becoming asset customer than age.
- However, age and experience,none of them show positive influence on dependent variable of our interest.
-Income has moderaly positive influence on becoming asset customer.
- Income has also postive impact on more expenditure on credit card

In [ ]:
# A function is created for stacked bar chart


def stacked_barplot(data, predictor, target):
    """
    Print the category counts and plot a stacked bar chart

    data: dataframe
    predictor: independent variable
    target: target variable
    """
    count = data[predictor].nunique()
    sorter = data[target].value_counts().index[-1]
    tab1 = pd.crosstab(data[predictor], data[target], margins=True).sort_values(
        by=sorter, ascending=False
    )
    print(tab1)
    print("-" * 120)
    tab = pd.crosstab(data[predictor], data[target], normalize="index").sort_values(
        by=sorter, ascending=False
    )
    tab.plot(kind="bar", stacked=True, figsize=(count + 5, 6))
    plt.legend(
        loc="lower left", frameon=False,
    )
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.show()

In [ ]:
stacked_barplot(loanc, "Education", "Personal_Loan")

It shows there are more asset customers with higher education than undergraduate. 

In [ ]:
stacked_barplot(loanc, "Online", "Personal_Loan")

No significant impact based on online presence on personal loan.

In [ ]:
stacked_barplot(loanc, "Securities_Account", "Personal_Loan")

There are little correlation having securities account create probility for personal loan account. 

In [ ]:
stacked_barplot(loanc, "CreditCard", "Personal_Loan")

There is no change having credit card or not on personal laon probability.

In [ ]:
stacked_barplot(loanc, "CD_Account", "Personal_Loan")

The customers who have CD Account shows more probability of being asset customer. 

In [ ]:
stacked_barplot(loanc, "Family", "Personal_Loan")

It shows clearly, as bigger the family size, more chances of becoming asset customer.

Data Decriptions:
- Most of the columns are integer and one is float
- there are no missing values in the data
- there are no duplicated values in the data
- there are some unrealistic negative values in the experience column
- there are outliers in CCAvg, Mortgage, Income
#Observations from exploratory data analysis and visualizations:
- Age and experience variables are highly correlated with another, it is better to drop one variable to avoid overfitting. 
- Income variable has postive influence on more expenditure on credit credit as well as on the dependent variable - personal loan.
- Very few percentage, below 10 percent of customers have personal loan, securities account, CD account.
- Presence on Internet, Credit card have no significant impact on having personal loan probability.
- Highly educated cusotmers have more probability of becoming personal loan customer.
- The customers, who have CD account, they also likely be the personal loan customers. 
- the customers have on average 20 years of working experience.
- 29 percent of customers are single
- Bigger family are more proned to be the personal loan customers
- From primary observations, bigger family size, highly educated, high income customers and the customers with cd account have more chances to be personal asset customer


# Data Cleaning

Based on the univariate analysis of variables, it is better to drop two columns at first, which seems to me unnecessary for the analysis, ID and ZIPCode. I could convert ZIPCode to State, however US have around 50 States and in this case, creating so many variable does not add so much value to our analysis.

In [ ]:
#Droping the column ID and ZIPCode
to_drop = ["ID", "ZIPCode", "Experience"]
loana.drop(to_drop, axis=1, inplace=True)
loana.head(10)

# Data Preparation

In [10]:
#Changing the category of ZIPcode
loanc["ZIPCode"]=loanc["ZIPCode"].astype("category")

In [12]:
#Dropping the experience column as it is perfectly correlated with age variable
loanc=loanc.drop("Experience", axis=1)

In [16]:
loanc.head()

,ID,Age,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,1,25,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,45,91330,4,1.0,2,0,0,0,0,0,1


In [19]:
pd.get_dummies(loanc, columns=["ZIPCode", "Education"], drop_first=True)

,ID,Age,Income,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,ZIPCode_90007,ZIPCode_90009,ZIPCode_90011,ZIPCode_90016,ZIPCode_90018,ZIPCode_90019,ZIPCode_90024,ZIPCode_90025,ZIPCode_90027,ZIPCode_90028,ZIPCode_90029,ZIPCode_90032,ZIPCode_90033,ZIPCode_90034,ZIPCode_90035,ZIPCode_90036,ZIPCode_90037,ZIPCode_90041,ZIPCode_90044,ZIPCode_90045,ZIPCode_90048,ZIPCode_90049,ZIPCode_90057,ZIPCode_90058,ZIPCode_90059,ZIPCode_90064,ZIPCode_90065,ZIPCode_90066,ZIPCode_90068,ZIPCode_90071,ZIPCode_90073,ZIPCode_90086,ZIPCode_90089,ZIPCode_90095,ZIPCode_90210,ZIPCode_90212,ZIPCode_90230,ZIPCode_90232,ZIPCode_90245,ZIPCode_90250,ZIPCode_90254,ZIPCode_90266,ZIPCode_90272,ZIPCode_90274,ZIPCode_90275,ZIPCode_90277,ZIPCode_90280,ZIPCode_90291,ZIPCode_90304,ZIPCode_90401,ZIPCode_90404,ZIPCode_90405,ZIPCode_90502,ZIPCode_90503,ZIPCode_90504,ZIPCode_90505,ZIPCode_90509,ZIPCode_90601,ZIPCode_90623,ZIPCode_90630,ZIPCode_90638,ZIPCode_90639,ZIPCode_90640,ZIPCode_90650,ZIPCode_90717,ZIPCode_90720,ZIPCode_90740,ZIPCode_90745,ZIPCode_90747,ZIPCode_90755,ZIPCode_90813,ZIPCode_90840,ZIPCode_91006,ZIPCode_91007,ZIPCode_91016,ZIPCode_91024,ZIPCode_91030,ZIPCode_91040,ZIPCode_91101,ZIPCode_91103,ZIPCode_91105,ZIPCode_91107,ZIPCode_91109,ZIPCode_91116,ZIPCode_91125,ZIPCode_91129,ZIPCode_91203,ZIPCode_91207,ZIPCode_91301,ZIPCode_91302,ZIPCode_91304,ZIPCode_91311,ZIPCode_91320,ZIPCode_91326,ZIPCode_91330,ZIPCode_91335,ZIPCode_91342,ZIPCode_91343,ZIPCode_91345,ZIPCode_91355,ZIPCode_91360,ZIPCode_91361,ZIPCode_91365,ZIPCode_91367,ZIPCode_91380,ZIPCode_91401,ZIPCode_91423,ZIPCode_91604,ZIPCode_91605,ZIPCode_91614,ZIPCode_91706,ZIPCode_91709,ZIPCode_91710,ZIPCode_91711,ZIPCode_91730,ZIPCode_91741,ZIPCode_91745,ZIPCode_91754,ZIPCode_91763,ZIPCode_91765,ZIPCode_91768,ZIPCode_91770,ZIPCode_91773,ZIPCode_91775,ZIPCode_91784,ZIPCode_91791,ZIPCode_91801,ZIPCode_91902,ZIPCode_91910,ZIPCode_91911,ZIPCode_91941,ZIPCode_91942,ZIPCode_91950,ZIPCode_92007,ZIPCode_92008,ZIPCode_92009,ZIPCode_92024,ZIPCode_92028,ZIPCode_92029,ZIPCode_92037,ZIPCode_92038,ZIPCode_92054,ZIPCode_92056,ZIPCode_92064,ZIPCode_92068,ZIPCode_92069,ZIPCode_92084,ZIPCode_92093,ZIPCode_92096,ZIPCode_92101,ZIPCode_92103,ZIPCode_92104,ZIPCode_92106,ZIPCode_92109,ZIPCode_92110,ZIPCode_92115,ZIPCode_92116,ZIPCode_92120,ZIPCode_92121,ZIPCode_92122,ZIPCode_92123,ZIPCode_92124,ZIPCode_92126,ZIPCode_92129,ZIPCode_92130,ZIPCode_92131,ZIPCode_92152,ZIPCode_92154,ZIPCode_92161,ZIPCode_92173,ZIPCode_92177,ZIPCode_92182,ZIPCode_92192,ZIPCode_92220,ZIPCode_92251,ZIPCode_92325,ZIPCode_92333,ZIPCode_92346,ZIPCode_92350,ZIPCode_92354,ZIPCode_92373,ZIPCode_92374,ZIPCode_92399,ZIPCode_92407,ZIPCode_92507,ZIPCode_92518,ZIPCode_92521,ZIPCode_92606,ZIPCode_92612,ZIPCode_92614,ZIPCode_92624,ZIPCode_92626,ZIPCode_92630,ZIPCode_92634,ZIPCode_92646,ZIPCode_92647,ZIPCode_92648,ZIPCode_92653,ZIPCode_92660,ZIPCode_92661,ZIPCode_92672,ZIPCode_92673,ZIPCode_92675,ZIPCode_92677,ZIPCode_92691,ZIPCode_92692,ZIPCode_92694,ZIPCode_92697,ZIPCode_92703,ZIPCode_92704,ZIPCode_92705,ZIPCode_92709,ZIPCode_92717,ZIPCode_92735,ZIPCode_92780,ZIPCode_92806,ZIPCode_92807,ZIPCode_92821,ZIPCode_92831,ZIPCode_92833,ZIPCode_92834,ZIPCode_92835,ZIPCode_92843,ZIPCode_92866,ZIPCode_92867,ZIPCode_92868,ZIPCode_92870,ZIPCode_92886,ZIPCode_93003,ZIPCode_93009,ZIPCode_93010,ZIPCode_93014,ZIPCode_93022,ZIPCode_93023,ZIPCode_93033,ZIPCode_93063,ZIPCode_93065,ZIPCode_93077,ZIPCode_93101,ZIPCode_93105,ZIPCode_93106,ZIPCode_93107,ZIPCode_93108,ZIPCode_93109,ZIPCode_93111,ZIPCode_93117,ZIPCode_93118,ZIPCode_93302,ZIPCode_93305,ZIPCode_93311,ZIPCode_93401,ZIPCode_93403,ZIPCode_93407,ZIPCode_93437,ZIPCode_93460,ZIPCode_93524,ZIPCode_93555,ZIPCode_93561,ZIPCode_93611,ZIPCode_93657,ZIPCode_93711,ZIPCode_93720,ZIPCode_93727,ZIPCode_93907,ZIPCode_93933,ZIPCode_93940,ZIPCode_93943,ZIPCode_93950,ZIPCode_93955,ZIPCode_94002,ZIPCode_94005,ZIPCode_94010,ZIPCode_94015,ZIPCode_94019,ZIPCode_94022,ZIPCode_94024,ZIPCode_94025,ZIPCode_94028,ZIPCode_9

# Splitting data

In [20]:
#Defining our independent and dependent variables
X=loanc.drop("Personal_Loan", axis=1)
y=loanc["Personal_Loan"]

In [24]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print(X_train.shape, X_test.shape)

(3500, 12) (1500, 12)


In [25]:
#Priting the number of rows in train and test data
print("Number of Rows in Train Data=", X_train.shape[0])
print("Number of Rows in Test Data=", X_test.shape[0])

Number of Rows in Train Data= 3500
Number of Rows in Test Data= 1500


In [26]:
print("Percentage of classes in training set:")
print(y_train.value_counts(normalize=True))
print("Percentage of classes in test set:")
print(y_test.value_counts(normalize=True))

Percentage of classes in training set:
0    0.905429
1    0.094571
Name: Personal_Loan, dtype: float64
Percentage of classes in test set:
0    0.900667
1    0.099333
Name: Personal_Loan, dtype: float64


# Logistic Regression

The confusion matrix

True Positives (TP): we correctly predicted that they will be customer 

True Negatives (TN): we correctly predicted they will not be loan custome 1339

False Positives (FP): we incorrectly predicted that they will be customer (a "Type I error") 12 Falsely predict positive Type I error

False Negatives (FN): we incorrectly predicted that they will not be customer (a "Type II error")  Falsely predict negative Type II error

# Model evaluation criterion
How could we make wrong predictions in our model:
- Predicting a customer will become personal loan customer, however the cusotmer will not be a persoanl loan customer. - Loss of resources

- Predicting a customer will not become personal loan customer, however in reality the customer could be a personal customer. - Loss of opportunity

- If we predict a customer who was going to contribute to the revenue as a customer who will not contribute to the revenue.
- In this case, we need to reduce False Negatives. 
- Recall should be maximized, the greater the recall higher the chances of minimizing the false negatives.

# Model Evaluation 

In [27]:
# defining a function to compute different metrics to check performance of a classification model built using sklearn


def model_performance_classification_sklearn_with_threshold(
    model, predictors, target, threshold=0.5
):
    """
    Function to compute different metrics, based on the threshold specified, to check classification model performance

    model: classifier
    predictors: independent variables
    target: dependent variable
    threshold: threshold for classifying the observation as class 1
    """

    # predicting using the independent variables
    pred_prob = model.predict_proba(predictors)[:, 1]
    pred_thres = pred_prob > threshold
    pred = np.round(pred_thres)

    acc = accuracy_score(target, pred)  # to compute Accuracy
    recall = recall_score(target, pred)  # to compute Recall
    precision = precision_score(target, pred)  # to compute Precision
    f1 = f1_score(target, pred)  # to compute F1-score

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {"Accuracy": acc, "Recall": recall, "Precision": precision, "F1": f1,},
        index=[0],
    )

    return df_perf

In [ ]:
##  Creating function to calculate recall score
def get_recall_score(model, predictors, target):
    """
    model: classifier
    predictors: independent variables
    target: dependent variable
    
    """
    prediction = model.predict(predictors)
    return recall_score(target, prediction)

In [ ]:
def confusion_matrix_sklearn(model, predictors, target):
    """
    To plot the confusion_matrix with percentages

    model: classifier
    predictors: independent variables
    target: dependent variable
    """
    y_pred = model.predict(predictors)
    cm = confusion_matrix(target, y_pred)
    labels = np.asarray(
        [
            ["{0:0.0f}".format(item) + "\n{0:.2%}".format(item / cm.flatten().sum())]
            for item in cm.flatten()
        ]
    ).reshape(2, 2)

    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=labels, fmt="")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [28]:
# Fit the model on train
model = LogisticRegression(solver="liblinear", random_state=1)
model.fit(X_train, y_train)
#predict on test
y_predict = model.predict(X_test)


coef_df = pd.DataFrame(model.coef_)
coef_df['intercept'] = model.intercept_
print(coef_df)

          0        1         2         3         4         5         6  \
0 -0.000036 -0.00008  0.036139 -0.000065  0.008662  0.005774  0.009381   

          7         8        9        10        11  intercept  
0  0.000858  0.000301  0.00268 -0.000067 -0.000006  -0.000036  


### Coefficient interpretation

* Coefficients of Age, Income, Family, CCAvg, CD_Account, and Education, and some levels of ZIPCode are positive an increase in these will lead to an increase in chances of taking a personal loan.

* Coefficients of Securities_Account, CreditCard, Online, and some levels of ZIPCode are negative an increase in these will lead to a decrease in chances of taking a personal loan.

Converting coefficients to odds
The coefficients of the logistic regression model are in terms of log(odd), to find the odds we have to take the exponential of the coefficients.
Therefore, odds = exp(b)
The percentage change in odds is given as odds = (exp(b) - 1) * 100

In [30]:
# converting coefficients to odds
odds = np.exp(model.coef_[0])

# finding the percentage change
perc_change_odds = (np.exp(model.coef_[0]) - 1) * 100

# removing limit from number of columns to display
pd.set_option("display.max_columns", None)

# adding the odds to a dataframe
pd.DataFrame({"Odds": odds, "Change_odd%": perc_change_odds}, index=X_train.columns).T

,ID,Age,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Securities_Account,CD_Account,Online,CreditCard
Odds,0.999964,0.999920,1.036800,0.999935,1.008700,1.005790,1.009425,1.000858,1.000301,1.002684,0.999933,0.999994
Change_odd%,-0.003597,-0.007958,3.680004,-0.006510,0.869982,0.579039,0.942481,0.085835,0.030145,0.268404,-0.006674,-0.000624


### Coefficient interpretations

* `Age`: Holding all other features constant a 1 unit increase in Age will increase the odds of a customer taking the personal loan by 1.004 times or a 0.48% increase in the odds.
* `Income`: Holding all other features constant a 1 unit increase in Income will increase the odds of taking a personal loan by 1.05 times or a 5.61% increase in the odds.
* `Family`: Holding all other features constant a 1 unit increase in Family will increase the odds of taking a personal loan by 1.88 times or a 88.01% increase in the odds.
* `CCAvg`: Holding all other features constant a 1 unit increase in CCAvg will increase the odds of a customer taking a personal loan by 1.20 times or a 20.47% increase in the odds.
* `Securities_Account`: The odds of a customer who has a Securities_Account with bank taking a personal loan is 0.45 times or 54.6% less than the customer who doesn't have a Securities_Account.
* `Online`: The odds of a customer who prefers internet banking facilities taking a personal loan is 0.58 times or 41.16% less than the customer who doesn't use internet banking facilities.
* The odds of a customer from ZIPCode_91 are 0.88 times or 11.16% less than the customer from ZIPCode_90. Similarly, the odds of a customer from ZIPCode_92 are 1.26 times or  26.26% more than the customer from ZIPCode_90. Interpretation of other ZIPCodes can be done in the same way by keeping ZIPCode_90 as the reference.
* Education and CD_Account have greater coefficients, so small increases in their value will have a bigger increase in chances of taking a personal loan.

`Interpretation for other attributes can be made similarly.`

In [32]:
model_score = model.score(X_test, y_test)
print(model_score)

0.9073333333333333


In [33]:
cm=metrics.confusion_matrix(y_test, y_predict, labels=[1, 0])

df_cm = pd.DataFrame(cm, index = [i for i in ["Actual 1"," Actual 0"]],
                  columns = [i for i in ["Predict 1","Predict 0"]])
plt.figure(figsize = (7,5))
sns.heatmap(df_cm, annot=True,fmt='g')
plt.show()

NameError: name 'metrics' is not defined

# Building decision tree model

We are builiding our model using the DecisionTreeClassifier function. We are using gini.

As our dependent variable frequency is only below 10 percent, the decision tree will be biased toward dominant class. 

Therefore, we are passing a dictionary {0:0.15,1:0.85} to the model to specify the weight of each class and the decision tree will give more weightage to class 1.

class_weight is a hyperparameter for the decision tree classifier.

In [ ]:
model = DecisionTreeClassifier(
    criterion="gini", class_weight={0: 0.15, 1: 0.85}, random_state=1
)

In [ ]:
model.fit(X_train, y_train)

# Checking model on train data

In [ ]:
confusion_matrix_sklearn(model, X_train, y_train)

In [ ]:
decision_tree_perf_train = get_recall_score(model, X_train, y_train)

print("Recall Score:", decision_tree_perf_train)

- Our model is able to classify with 0 failure. Model is able to perfectly classify all the data points on the training set.
- It might lead to ovefitting, as we generally know performin well on train data, will not perform well on test data

# Checking performence on test set

In [ ]:
confusion_matrix_sklearn(model, X_test, y_test)

In [ ]:
decision_tree_perf_test = get_recall_score(model, X_test, y_test)
print("Recall Score:", decision_tree_perf_test)

There is moderate gap between scores of train data and test data. There are scopes to improve.

# Visualizing decision tree

In [ ]:
#creating a list of column names
feature_names = X_train.columns.to_list()

In [ ]:
plt.figure(figsize=(20, 30))
out = tree.plot_tree(
    model,
    feature_names=feature_names,
    filled=True,
    fontsize=9,
    node_ids=False,
    class_names=None,
)
# below code will add arrows to the decision tree split if they are missing
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor("black")
        arrow.set_linewidth(1)
plt.show()

In [ ]:
# Text report showing the rules of a decision tree -

print(tree.export_text(model, feature_names=feature_names, show_weights=True))

The tree is difficult to explain becuase of the complexity of the tree.

In [ ]:
# Showing the importance of features in tree building also known as gini importance

print(
    pd.DataFrame(
        model.feature_importances_, columns=["Imp"], index=X_train.columns
    ).sort_values(by="Imp", ascending=False)
)

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12, 12))
plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color="violet", align="center")
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel("Relative Importance")
plt.show()

As we predicted before from our EDA Analysis, the tree importance features also depict the similar pictures, the importance of the variables like, income, family, education, CCAvg and CD Account

# Reducing overfitting

- we will use Gridsearch for hyperparameter tuning of the tree model
-  it is a exhaustive search optimized by using cross validation grid search

In [ ]:
# Choosing the type of classifier.
estimator = DecisionTreeClassifier(random_state=1, class_weight={0: 0.15, 1: 0.85})

# Grid of parameters to choose from
parameters = {
    "max_depth": [5, 10, 15, None],
    "criterion": ["entropy", "gini"],
    "splitter": ["best", "random"],
    "min_impurity_decrease": [0.00001, 0.0001, 0.01],
}

# scoring type is specified to compare parameter combinations
scorer = make_scorer(recall_score)

# Running the grid search
grid_obj = GridSearchCV(estimator, parameters, scoring=scorer, cv=5)
grid_obj = grid_obj.fit(X_train, y_train)

# Setting the clf to the best combination of parameters
estimator = grid_obj.best_estimator_

# Fitting the best algorithm to the data.
estimator.fit(X_train, y_train)

# Checking performence on training set

In [ ]:
confusion_matrix_sklearn(estimator, X_train, y_train)

In [ ]:
decision_tree_tune_perf_train = get_recall_score(estimator, X_train, y_train)
print("Recall Score:", decision_tree_tune_perf_train)

# Checking performence on test set

In [ ]:
confusion_matrix_sklearn(estimator, X_test, y_test)

In [ ]:
decision_tree_tune_perf_test = get_recall_score(estimator, X_test, y_test)
print("Recall Score:", decision_tree_tune_perf_test)

The model is giving better result now than previous one and near to almost a very good one. 

# Visualizing the decision Tree

In [ ]:
plt.figure(figsize=(15, 10))
out = tree.plot_tree(
    estimator,
    feature_names=feature_names,
    filled=True,
    fontsize=9,
    node_ids=False,
    class_names=None,
)
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor("black")
        arrow.set_linewidth(1)
plt.show()

In [ ]:
# Text report showing the rules of a decision tree -

print(tree.export_text(estimator, feature_names=feature_names, show_weights=True))

# Observations from the tree
-as a example, if income of one customer is 128 thousand, with family size 2.41 and education 1.47, customer may become the personal loan customer

In [ ]:
#Showing the gini importance of features

print(
    pd.DataFrame(
        estimator.feature_importances_, columns=["Imp"], index=X_train.columns
    ).sort_values(by="Imp", ascending=False)
)

In [ ]:
importances = estimator.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12, 12))
plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color="violet", align="center")
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel("Relative Importance")
plt.show()

- We see the importance of education and Credit card expenditaure has improved, where the importance of income and family little bit decreased.

# Cost Complexity Pruning
Cost complexity pruning provides a option to control the size of a tree.This  technique is parameterized by the cost complexity parameter, ccp_alpha. Greater values of ccp_alpha increase the number of nodes pruned. 

# facing technicla problem with cost complexity pruning

In [ ]:
clf = DecisionTreeClassifier(random_state=1, class_weight={0: 0.15, 1: 0.85})
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")
plt.show()

# Comparing all decision tree models

In [ ]:
# training performance comparison

models_train_comp_df = pd.DataFrame(
    [
        decision_tree_perf_train,
        decision_tree_tune_perf_train,
    ],
    columns=["Recall on training set"],
)

print("Training performance comparison:")
models_train_comp_df

In [ ]:
# testing performance comparison

models_test_comp_df = pd.DataFrame(
    [
        decision_tree_perf_test,
        decision_tree_tune_perf_test,
    ],
    columns=["Recall on testing set"],
)

print("Test performance comparison:")
models_test_comp_df

#the model before pruning has given the best performence on training data. However as you know, that will not be good enought o generalize with this model, which we have better result after doing the prepruning, performence of train and test data are more close, better to geralize.

- As to develop the insights for the bank Alllife, we have gone through Logistic Regression, confusion matrix, decision tree, decision tree with preprunning
- After doing these experiments, we have found some insightfuls informations from the data
- As important variables, we can identify, income of the customer, education, family and CCAvg-expenditure on credit cards, and CD Account holders

Recommendations for the Bank:

- This Bank should more focus on profile of the customer who has higher income people.
- Secondly, the family with more members also tends to have personal loan. They should consider the bigger family, and also to attract them, they should bring the orientation in their promotion material about the family freinliness of the bank, education loan, childrens benefits loan, or joint account for family etc. 
- As we also see, there is more probability of higher educated people to take personal loan, the bank also should focus on this segement of the market, by providing loan for students to continue their studies, higher studies etc.
- It is also observed, the customer who has more expenditure using credit card, they also a possible target customer for the banks future loan business.
- Also the customer who have CD Account are also personal loan customer, they could be targeted more for personal loan 
